<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_tweaked_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_tweaked.py

4‐bit parity classification with an enhanced metric-learning stage:
- analytic CNOT baseline
- variational QNN baseline
- quantum metric learning with diagnostics
- final SVM CV
- kernel and histogram visualizations
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Hyperparameters
# -----------------------------------------------------------------------------
n_bits       = 4
qnn_epochs   = 30
qnn_lr       = 0.4
metric_steps = 300
metric_lr    = 0.1
svm_C        = 1.0
seed         = 42
num_layers   = 2     # number of rotational‐entanglement layers

np.random.seed(seed)
pnp.random.seed(seed)

# -----------------------------------------------------------------------------
# 1) Parity dataset (scaled & centered)
# -----------------------------------------------------------------------------
X_bits   = np.array(list(itertools.product([0, 1], repeat=n_bits)))
# scale bits to {0, π}
X_feat   = X_bits.astype(float) * np.pi
y_bits   = np.sum(X_bits, axis=1) % 2
y_signed = 1 - 2 * y_bits  # +1 even, -1 odd

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# -----------------------------------------------------------------------------
# 2) Analytic CNOT baseline
# -----------------------------------------------------------------------------
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_exact = [0 if analytic_circuit(x) > 0 else 1 for x in X_bits]
acc_exact   = accuracy_score(y_bits, preds_exact)
print(f"Analytic CNOT Cascade Accuracy: {acc_exact:.3f}")

# -----------------------------------------------------------------------------
# 3) Variational QNN baseline (5‐fold CV)
# -----------------------------------------------------------------------------
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    # embed input as π rotations
    for i, angle in enumerate(x):
        qml.RY(angle, wires=i)
    # trainable rotations
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    # ladder entanglement
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

var_scores = []
for tr, te in kf.split(X_feat, y_bits):
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)

    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X_feat[i]) for i in tr]
            return pnp.mean((pnp.array(preds) - y_signed[tr]) ** 2)
        params = opt.step(cost, params)

    preds_test = [qnn_circuit(params, X_feat[i]) for i in te]
    bits_pred  = [0 if v > 0 else 1 for v in preds_test]
    var_scores.append(accuracy_score(y_bits[te], bits_pred))

print(f"Variational QNN CV Accuracy: {np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}")

# -----------------------------------------------------------------------------
# 4) Enhanced Quantum Metric Learning with Adam
# -----------------------------------------------------------------------------
dev_feat = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_feat, interface="autograd")
def feature_map(params, x):
    # angle embedding
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    offset = 0
    # multi-layer Rot + ring‐entangle
    for layer in range(num_layers):
        for i in range(n_bits):
            phi, theta, omega = params[offset + 3*i : offset + 3*i + 3]
            qml.Rot(phi, theta, omega, wires=i)
        for i in range(n_bits):
            qml.CNOT(wires=[i, (i + 1) % n_bits])
        offset += 3 * n_bits
    # final single‐qubit Z,Y rotations
    for i in range(n_bits):
        qml.RZ(params[offset + i], wires=i)
        qml.RY(params[offset + n_bits + i], wires=i)
    return qml.state()

def build_kernel_matrix(params):
    states = pnp.stack([feature_map(params, x) for x in X_feat])
    re, im  = pnp.real(states), pnp.imag(states)
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    return real_ov**2 + imag_ov**2

def centered_alignment(K):
    N  = K.shape[0]
    H  = pnp.eye(N) - pnp.ones((N, N)) / N
    Kc = H @ K @ H
    T  = pnp.outer(y_signed, y_signed)
    num = pnp.sum(Kc * T)
    den = pnp.linalg.norm(Kc) * pnp.linalg.norm(T)
    return num / den

# initialize parameters
dim_params = num_layers * 3 * n_bits + 2 * n_bits
params_k   = pnp.random.randn(dim_params, requires_grad=True) * 0.1
opt_metric = qml.AdamOptimizer(stepsize=metric_lr)

print("\nTraining feature map via centered alignment (Adam)")
for step in range(metric_steps):
    # compute loss and gradient norm
    def loss_fn(p):
        K = build_kernel_matrix(p)
        return -centered_alignment(K)

    # fetch gradient to inspect magnitude
    grad_fn     = qml.grad(loss_fn)
    grads       = grad_fn(params_k)
    grad_norm   = np.linalg.norm(grads)
    params_k, loss_val = opt_metric.step_and_cost(loss_fn, params_k)

    if step % 50 == 0 or step == metric_steps - 1:
        align_val = centered_alignment(build_kernel_matrix(params_k))
        print(f" Step {step:3d}: loss {loss_val:.4f}, grad norm {grad_norm:.4f}, alignment {align_val:.4f}")

# -----------------------------------------------------------------------------
# 5) Final SVM CV on learned kernel
# -----------------------------------------------------------------------------
K_learned = np.array(build_kernel_matrix(params_k))
ker_scores = []
for tr, te in kf.split(K_learned, y_bits):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_learned[np.ix_(tr, tr)], y_bits[tr])
    preds = svc.predict(K_learned[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y_bits[te], preds))

print(f"\nLearned Kernel CV Accuracy: {np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}")

# -----------------------------------------------------------------------------
# 6) Kernel & Histogram Visualizations
# -----------------------------------------------------------------------------
# Heatmap of learned kernel
plt.figure(figsize=(5, 5))
plt.title("Learned Kernel |⟨ψ_i|ψ_j⟩|²")
plt.imshow(K_learned, cmap="viridis", origin="lower")
plt.colorbar(label="K value")
plt.xlabel("j")
plt.ylabel("i")
plt.tight_layout()

# Histogram of kernel entries
plt.figure(figsize=(5, 3))
plt.title("Kernel Value Distribution")
vals = K_learned.flatten()
plt.hist(vals, bins=30, color="navy", alpha=0.7)
plt.xlabel("Kernel entry")
plt.ylabel("Frequency")
plt.tight_layout()

plt.show()